In [6]:
import os
import sqlite3
import pandas as pd
import numpy as np
from tqdm.auto import tqdm
from typing import List, Dict, Optional, Tuple, Union
from sentence_transformers import SentenceTransformer
from sklearn.metrics.pairwise import cosine_similarity
import re
import string
from collections import Counter
import warnings
warnings.filterwarnings('ignore')

In [7]:
# Configuration
class Config:
    DB_PATH = "qna_database.db"
    EMBEDDING_MODEL = "all-MiniLM-L6-v2"
    CHUNK_SIZE = 500  # For batch processing
    VECTOR_DIM = 384  # Dimension for all-MiniLM-L6-v2 embeddings
    MIN_WORD_LENGTH = 3  # Minimum word length for keyword processing
    STOPWORDS = {
        'the', 'and', 'of', 'in', 'to', 'a', 'is', 'for', 'on', 'that', 'it', 'with', 'as', 'be', 'by', 'this', 'are', 'at'
    }

In [14]:
import sqlite3
import re
from typing import List, Dict, Tuple, Optional
from collections import Counter
import numpy as np
import pandas as pd
from tqdm import tqdm
from sentence_transformers import SentenceTransformer
from sklearn.metrics.pairwise import cosine_similarity

class QnADatabase:
    def __init__(self, db_path: str = Config.DB_PATH):
        """Initialize with optimized SQLite settings"""
        self.db_path = db_path
        self.conn = None
        self.embedding_model = None
        self._initialize_db()
        
    def _initialize_db(self):
        """Create database with optimized schema"""
        self.conn = sqlite3.connect(self.db_path, timeout=30)
        self.conn.execute("PRAGMA journal_mode = WAL")
        self.conn.execute("PRAGMA synchronous = NORMAL")
        self.conn.execute("PRAGMA cache_size = -100000")  # 100MB cache
        
        # Main Q&A table with additional metadata fields
        self.conn.execute("""
        CREATE TABLE IF NOT EXISTS qna_pairs (
            id INTEGER PRIMARY KEY AUTOINCREMENT,
            question TEXT NOT NULL,
            answer TEXT NOT NULL,
            category TEXT,
            word_count INTEGER,
            created_at TIMESTAMP DEFAULT CURRENT_TIMESTAMP,
            last_accessed TIMESTAMP,
            usage_count INTEGER DEFAULT 0,
            keywords TEXT,
            normalized_question TEXT
        )""")
        
        # Vector embeddings table
        self.conn.execute(f"""
        CREATE TABLE IF NOT EXISTS qna_embeddings (
            qna_id INTEGER PRIMARY KEY,
            question_vector BLOB,
            answer_vector BLOB,
            FOREIGN KEY (qna_id) REFERENCES qna_pairs(id)
        )""")
        
        # Create indexes
        self.conn.execute("CREATE INDEX IF NOT EXISTS idx_category ON qna_pairs(category)")
        self.conn.execute("CREATE INDEX IF NOT EXISTS idx_word_count ON qna_pairs(word_count)")
        self.conn.execute("CREATE INDEX IF NOT EXISTS idx_normalized_question ON qna_pairs(normalized_question)")
        
        # Full-text search with additional configuration
        self.conn.execute("""
        CREATE VIRTUAL TABLE IF NOT EXISTS qna_search 
        USING fts5(question, answer, keywords, tokenize='porter unicode61')
        """)

    def _get_embedding_model(self):
        """Get better embedding model"""
        if self.embedding_model is None:
            # Consider using a larger model for better accuracy
            self.embedding_model = SentenceTransformer('all-mpnet-base-v2')  # Better than default
        return self.embedding_model

    def _text_to_vector(self, text: str) -> bytes:
        """Convert text to compressed vector"""
        model = self._get_embedding_model()
        vector = model.encode(text)
        return vector.tobytes()

    def _vector_to_array(self, blob: bytes) -> np.ndarray:
        """Convert blob back to numpy array"""
        return np.frombuffer(blob, dtype=np.float32)
    
    def _normalize_text(self, text: str) -> str:
        """Normalize text for exact matching"""
        text = text.lower().strip()
        text = re.sub(r'[^\w\s]', '', text)  # Remove punctuation
        return text
    
    def _extract_keywords(self, text: str) -> str:
        """Extract important keywords from text"""
        # Remove punctuation and convert to lowercase
        text = text.lower()
        text = re.sub(r'[^\w\s]', '', text)
        
        # Tokenize and filter
        words = text.split()
        words = [word for word in words 
                if len(word) >= Config.MIN_WORD_LENGTH 
                and word not in Config.STOPWORDS]
        
        # Count word frequencies and get most common
        word_counts = Counter(words)
        keywords = [word for word, count in word_counts.most_common(10)]  # Top 10 keywords
        
        return ' '.join(keywords)

    def batch_insert(self, qna_list: List[Dict]):
        """Optimized bulk insert with embeddings and keyword processing"""
        if not qna_list:
            return
            
        with self.conn:
            cursor = self.conn.cursor()
            
            # Check for existing questions to prevent duplicates using normalized version
            existing_questions = set()
            cursor.execute("SELECT normalized_question FROM qna_pairs")
            for row in cursor.fetchall():
                existing_questions.add(row[0])
            
            # Process and filter out duplicates
            processed_qna = []
            for qna in qna_list:
                norm_question = self._normalize_text(qna['question'])
                if norm_question not in existing_questions:
                    # Extract keywords from both question and answer
                    question_keywords = self._extract_keywords(qna['question'])
                    answer_keywords = self._extract_keywords(qna['answer'])
                    combined_keywords = f"{question_keywords} {answer_keywords}"
                    
                    processed_qna.append({
                        'question': qna['question'],
                        'answer': qna['answer'],
                        'category': qna.get('category'),
                        'word_count': len(qna['answer'].split()),
                        'keywords': combined_keywords,
                        'normalized_question': norm_question
                    })
                    existing_questions.add(norm_question)
            
            if not processed_qna:
                print("No new Q&A pairs to insert")
                return
            
            # Insert only unique Q&A pairs with additional metadata
            cursor.executemany("""
            INSERT INTO qna_pairs (question, answer, category, word_count, keywords, normalized_question)
            VALUES (?, ?, ?, ?, ?, ?)
            """, [(q['question'], q['answer'], q['category'], q['word_count'], 
                  q['keywords'], q['normalized_question']) for q in processed_qna])
            
            # Get inserted IDs
            cursor.execute("SELECT last_insert_rowid() - ? + 1, last_insert_rowid()", (len(processed_qna),))
            first_id, last_id = cursor.fetchone()
            
            # Generate and store embeddings
            for i in tqdm(range(len(processed_qna)), desc="Generating embeddings"):
                qna = processed_qna[i]
                q_vector = self._text_to_vector(qna['question'])
                a_vector = self._text_to_vector(qna['answer'])
                cursor.execute("""
                INSERT INTO qna_embeddings (qna_id, question_vector, answer_vector)
                VALUES (?, ?, ?)
                """, (first_id + i, q_vector, a_vector))
            
            # Update full-text search index with keywords
            cursor.executemany("""
            INSERT INTO qna_search (question, answer, keywords)
            VALUES (?, ?, ?)
            """, [(q['question'], q['answer'], q['keywords']) for q in processed_qna])
    
    def semantic_search(self, query: str, top_k: int = 5, threshold: float = 0.15) -> List[Tuple[str, str]]:
        """Context-aware semantic search"""
        cursor = self.conn.cursor()
        
        query_vec = self._text_to_vector(query)
        query_arr = self._vector_to_array(query_vec)
        
        # Get all stored embeddings with their keywords
        cursor.execute("""
        SELECT qna_pairs.question, qna_pairs.answer, 
               qna_embeddings.question_vector, qna_embeddings.answer_vector,
               qna_pairs.keywords
        FROM qna_pairs
        JOIN qna_embeddings ON qna_pairs.id = qna_embeddings.qna_id
        """)
        
        results = []
        
        for question, answer, q_vec_blob, a_vec_blob, keywords in cursor.fetchall():
            q_vec = self._vector_to_array(q_vec_blob)
            a_vec = self._vector_to_array(a_vec_blob)
            
            # Calculate base similarities
            q_sim = cosine_similarity([query_arr], [q_vec])[0][0]
            a_sim = cosine_similarity([query_arr], [a_vec])[0][0]
            
            # Calculate keyword similarity if keywords exist
            kw_sim = 0
            if keywords:
                kw_vec = self._text_to_vector(keywords)
                kw_sim = cosine_similarity([query_arr], [self._vector_to_array(kw_vec)])[0][0]
            
            # Contextual weighting:
            # 1. Higher weight if query terms appear in the question
            query_terms = set(self._normalize_text(query).split())
            question_terms = set(self._normalize_text(question).split())
            term_overlap = len(query_terms & question_terms) / len(query_terms) if query_terms else 0
            
            # 2. Higher weight if the answer is concise (word_count < 50)
            word_count = len(answer.split())
            conciseness = 0.1 if word_count < 50 else 0  # Small bonus for concise answers
            
            # Combined similarity score
            similarity = (
                0.5 * q_sim +  # Question similarity
                0.3 * a_sim +  # Answer similarity
                0.1 * kw_sim +  # Keyword similarity
                0.1 * term_overlap +  # Term overlap bonus
                conciseness  # Conciseness bonus
            )
            
            if similarity >= threshold:
                results.append((question, answer, similarity))
        
        # Sort and deduplicate
        unique_results = {}
        for question, answer, score in sorted(results, key=lambda x: x[2], reverse=True):
            norm_q = self._normalize_text(question)
            if norm_q not in unique_results or score > unique_results[norm_q][2]:
                unique_results[norm_q] = (question, answer, score)
        
        sorted_results = sorted(unique_results.values(), key=lambda x: x[2], reverse=True)
        return [(q, a) for q, a, _ in sorted_results[:top_k]]
        
    def keyword_search(self, query: str, limit: int = 5) -> List[Tuple[str, str]]:
        """Precision keyword search that matches terms and evaluates context"""
        cursor = self.conn.cursor()
        
        # Normalize and extract meaningful terms
        query = self._normalize_text(query)
        query_terms = [term for term in query.split() 
                      if len(term) >= Config.MIN_WORD_LENGTH 
                      and term not in Config.STOPWORDS]
        
        if not query_terms:
            return []
        
        # Get all potential matches from the database
        all_qna = []
        cursor.execute("""
        SELECT question, answer, keywords 
        FROM qna_pairs
        """)
        all_qna = cursor.fetchall()
        
        # Score each potential match
        scored_results = []
        for question, answer, keywords in all_qna:
            # Combine question, answer and keywords for matching
            content = f"{question} {answer} {keywords}".lower()
            norm_question = self._normalize_text(question)
            
            # Exact match scoring
            exact_matches = sum(
                1 for term in query_terms 
                if f" {term} " in f" {content} "
            )
            
            # Partial match scoring (substring matches)
            partial_matches = sum(
                1 for term in query_terms 
                if term in content
            )
            
            # Positional scoring - terms in question get higher weight
            question_terms = sum(
                2 for term in query_terms  # Higher weight for question matches
                if term in norm_question
            )
            
            # Combine scores with weighting
            score = (
                (exact_matches * 3) +  # Strong weight for exact matches
                (partial_matches * 1) +  # Lower weight for partial matches
                question_terms  # Additional weight for question matches
            )
            
            if score > 0:
                scored_results.append((question, answer, score))
        
        # Sort by score and remove duplicates
        unique_results = {}
        for question, answer, score in sorted(scored_results, key=lambda x: x[2], reverse=True):
            norm_q = self._normalize_text(question)
            if norm_q not in unique_results or score > unique_results[norm_q][2]:
                unique_results[norm_q] = (question, answer, score)
        
        # Return top results
        sorted_results = sorted(unique_results.values(), key=lambda x: x[2], reverse=True)
        return [(q, a) for q, a, _ in sorted_results[:limit]]

    def hybrid_search(self, query: str, top_k: int = 5) -> List[Tuple[str, str]]:
        """Intelligent hybrid search that combines best approaches"""
        # First get keyword matches
        keyword_results = self.keyword_search(query, top_k*2)  # Get extra for filtering
        
        # Then get semantic matches
        semantic_results = self.semantic_search(query, top_k*2)
        
        # Combine results with priority to exact keyword matches
        combined = []
        seen = set()
        
        # Add keyword matches first if they have high term overlap
        for q, a in keyword_results:
            norm_q = self._normalize_text(q)
            if norm_q not in seen:
                # Check if the question contains most query terms
                query_terms = set(self._normalize_text(query).split())
                question_terms = set(norm_q.split())
                overlap = len(query_terms & question_terms) / len(query_terms) if query_terms else 0
                
                if overlap > 0.5:  # At least 50% term overlap
                    combined.append((q, a, 1.0))  # High confidence
                    seen.add(norm_q)
        
        # Add semantic matches if we need more results
        for q, a in semantic_results:
            norm_q = self._normalize_text(q)
            if norm_q not in seen and len(combined) < top_k:
                combined.append((q, a, 0.8))  # Slightly lower confidence
                seen.add(norm_q)
        
        # Final sort by confidence and return
        combined.sort(key=lambda x: x[2], reverse=True)
        return [(q, a) for q, a, _ in combined[:top_k]]
    
    def _expand_query(self, query: str) -> str:
        """Expand query with related terms/synonyms"""
        # Simple implementation - add synonyms for common terms
        synonym_map = {
            'wallet': 'wallet address cryptocurrency',
            'miner': 'mining validator blockchain',
            'p2p': 'peer-to-peer decentralized',
        }
        
        expanded = []
        for term in query.lower().split():
            if term in synonym_map:
                expanded.append(synonym_map[term])
            else:
                expanded.append(term)
        
        return ' '.join(expanded) if expanded else query

    def get_all_data(self, limit: Optional[int] = None) -> pd.DataFrame:
        """Export all data with optional limit"""
        query = "SELECT * FROM qna_pairs"
        if limit:
            query += f" LIMIT {limit}"
        return pd.read_sql(query, self.conn)

    def optimize(self):
        """Database maintenance"""
        print("Optimizing database...")
        self.conn.execute("VACUUM")
        self.conn.execute("ANALYZE")
        self.conn.execute("PRAGMA optimize")

    def close(self):
        """Clean up resources"""
        if self.conn:
            self.conn.close()
        if self.embedding_model:
            del self.embedding_model

In [15]:
# Data Processing Utilities
class DataProcessor:
    @staticmethod
    def parse_text_file(file_path: str, question_prefix: str = "Q:", answer_prefix: str = "A:") -> List[Dict]:
        """Improved text file parser with better error handling"""
        qna_pairs = []
        current_q = None
        current_a = []
        
        try:
            with open(file_path, 'r', encoding='utf-8') as f:
                for line in f:
                    line = line.strip()
                    if not line:  # Skip empty lines
                        continue
                        
                    if line.startswith(question_prefix):
                        if current_q is not None:
                            qna_pairs.append({
                                "question": current_q,
                                "answer": "\n".join(current_a).strip()
                            })
                        current_q = line[len(question_prefix):].strip()
                        current_a = []
                    elif line.startswith(answer_prefix):
                        current_a.append(line[len(answer_prefix):].strip())
                    elif current_a:  # Only add to answer if we're in an answer block
                        current_a.append(line)
                
                # Add the last pair if it exists
                if current_q is not None:
                    qna_pairs.append({
                        "question": current_q,
                        "answer": "\n".join(current_a).strip()
                    })
        except Exception as e:
            print(f"Error processing file {file_path}: {str(e)}")
            return []
            
        return qna_pairs

    @staticmethod
    def chunk_list(lst: List, chunk_size: int):
        """Yield successive chunk_size chunks from lst"""
        for i in range(0, len(lst), chunk_size):
            yield lst[i:i + chunk_size]

In [16]:
def main():
    # Initialize database with cleanup
    if os.path.exists(Config.DB_PATH):
        os.remove(Config.DB_PATH)
    db = QnADatabase()
    
    # Sample data for demonstration
    sample_data = [
        {
            "question": "What is P2P? ",
            "answer": "A Peer-to-Peer (P2P) payment system, seamlessly integrated with blockchain technology, a decentralized application (DApp), and MetaMask wallet, orchestrates a streamlined and secure process for transparent transactions among users."
        },
        {
            "question": "Compare and Contrast Private and Public Key: ",
            "answer": "The private key allows you to have access to your funds through the crypto wallet. it is used to send Bitcoin and must be protected and secured. As for the public key, it is used to receive Bitcoin and can be published anywhere safely."
        }
    ]
    
    # Auto-categorize questions
    def detect_category(question: str) -> str:
        question_lower = question.lower()
        if 'p2p' in question_lower or 'peer-to-peer' in question_lower:
            return "networking"
        elif 'private key' in question_lower or 'public key' in question_lower:
            return "security"
        elif 'blockchain' in question_lower:
            return "fundamentals"
        elif 'proof of work' in question_lower or 'pow' in question_lower:
            return "consensus"
        elif 'wallet' in question_lower:
            return "wallets"
        elif 'smart contract' in question_lower:
            return "development"
        else:
            return "general"
    
    # Add categories to sample data
    for item in sample_data:
        item["category"] = detect_category(item["question"])
    
    # Process and insert data
    print("Inserting sample data...")
    db.batch_insert(sample_data)
    
    # For large files
    try:
        file_path = "/kaggle/input/db-19-txt"
        if os.path.exists(file_path):
            print("Processing large file...")
            qna_pairs = DataProcessor.parse_text_file(file_path)
            
            # Auto-categorize parsed questions
            for item in qna_pairs:
                item["category"] = detect_category(item["question"])
            
            print(f"Processing {len(qna_pairs)} Q&A pairs...")
            for chunk in DataProcessor.chunk_list(qna_pairs, Config.CHUNK_SIZE):
                db.batch_insert(chunk)
    except Exception as e:
        print(f"Error processing large file: {str(e)}")
        import traceback
        traceback.print_exc()
    
    # Test with new questions not in the sample data
    test_questions = [
        "What are the advantages of P2P networks?",
        "Explain the difference between hot and cold wallets",
        "What is the role of miners in blockchain?"
    ]
    
    print("\n" + "="*50)
    print("Testing with new questions not in sample data")
    print("="*50)
    
    for question in test_questions:
        print(f"\nQuestion: '{question}'")
        
        # Semantic search
        print("\nSemantic search results:")
        semantic_results = db.semantic_search(question)
        if semantic_results:
            for i, (q, a) in enumerate(semantic_results, 1):
                print(f"{i}. Question: {q}")
                print(f"   Answer: {a}")
                print(f"   {'-'*50}")
        else:
            print("No semantic matches found")
        
        # Keyword search
        print("\nKeyword search results:")
        keyword_results = db.keyword_search(question)
        if keyword_results:
            for i, (q, a) in enumerate(keyword_results, 1):
                print(f"{i}. Question: {q}")
                print(f"   Answer: {a}")
                print(f"   {'-'*50}")
        else:
            print("No keyword matches found")
    
    # Export data with categories
    df = db.get_all_data(limit=10)
    print("\nSample data from database:")
    print(df[['question', 'category']].head())
    
    # Show category distribution
    print("\nCategory distribution:")
    print(df['category'].value_counts())
    
    # Maintenance
    db.optimize()
    db.close()

if __name__ == "__main__":
    main()

Inserting sample data...


Generating embeddings:   0%|          | 0/2 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Generating embeddings:  50%|█████     | 1/2 [00:02<00:02,  2.92s/it]

...

1. Question: Compare and Contrast Private and Public Key: 
   Answer: The private key allows you to have access to your funds through the crypto wallet. it is used to send Bitcoin and must be protected and secured. As for the public key, it is used to receive Bitcoin and can be published anywhere safely.
   --------------------------------------------------
2. Question: What is P2P? 
   Answer: A Peer-to-Peer (P2P) payment system, seamlessly integrated with blockchain technology, a decentralized application (DApp), and MetaMask wallet, orchestrates a streamlined and secure process for transparent transactions among users.
   --------------------------------------------------

Keyword search results:
No keyword matches found

Question: 'What is the role of miners in blockchain?'

Semantic search results:


Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

1. Question: What is P2P? 
   Answer: A Peer-to-Peer (P2P) payment system, seamlessly integrated with blockchain technology, a decentralized application (DApp), and MetaMask wallet, orchestrates a streamlined and secure process for transparent transactions among users.
   --------------------------------------------------
2. Question: Compare and Contrast Private and Public Key: 
   Answer: The private key allows you to have access to your funds through the crypto wallet. it is used to send Bitcoin and must be protected and secured. As for the public key, it is used to receive Bitcoin and can be published anywhere safely.
   --------------------------------------------------

Keyword search results:
1. Question: What is P2P? 
   Answer: A Peer-to-Peer (P2P) payment system, seamlessly integrated with blockchain technology, a decentralized application (DApp), and MetaMask wallet, orchestrates a streamlined and secure process for transparent transactions among users.
   ------------------